# 4. Unsupervised Model Development

In this notebook, we will develop and train the unsupervised models for artifact removal. We will focus on the Autoencoder, as it's a powerful method for learning a compressed representation of the signal and reconstructing it without noise.

In [ ]:
import numpy as np
import pandas as pd
import sys
from sklearn.preprocessing import MinMaxScaler

# Add src to path to import custom modules
sys.path.append('../src')

from models.autoencoder import build_autoencoder
from visualization.plot import plot_eda_comparison

# Placeholder: Load your segmented, preprocessed data here
# For now, we generate synthetic 'clean' and 'artifact' data
def generate_data(n_samples, length):
    # Generate 'clean' sine waves
    clean_data = np.sin(np.linspace(0, 10, length) * np.random.uniform(1, 3, (n_samples, 1)))
    # Create noisy data by adding artifacts
    noisy_data = clean_data.copy()
    for i in range(n_samples):
        start = np.random.randint(0, length - 100)
        noisy_data[i, start:start+50] += np.random.normal(0, 0.5, 50)
    return noisy_data, clean_data

SEGMENT_LENGTH = 700
noisy_train, clean_train = generate_data(1000, SEGMENT_LENGTH)
noisy_test, clean_test = generate_data(200, SEGMENT_LENGTH)

# Reshape for Conv1D: (samples, timesteps, features)
X_train = noisy_train[..., np.newaxis]
y_train = clean_train[..., np.newaxis]
X_test = noisy_test[..., np.newaxis]
y_test = clean_test[..., np.newaxis]

## 4.1 Build and Train the Autoencoder

In [ ]:
autoencoder = build_autoencoder((SEGMENT_LENGTH, 1))

history = autoencoder.fit(
    X_train,
    X_train, # Autoencoders learn to reconstruct their own input
    epochs=50,
    batch_size=32,
    shuffle=True,
    validation_data=(X_test, X_test)
)

# Save the model
autoencoder.save('../models/autoencoder.h5')

## 4.2 Evaluate Reconstruction

Let's see how well the autoencoder reconstructs a noisy signal from the test set.

In [ ]:
reconstructed_signals = autoencoder.predict(X_test)

# Compare a sample
sample_index = 10
raw_signal = X_test[sample_index].flatten()
reconstructed_signal = reconstructed_signals[sample_index].flatten()

plot_eda_comparison(raw_signal, {'Autoencoder': reconstructed_signal}, title='Autoencoder Reconstruction')